# LLM for alle - Introduksjonskurs til språkmodeller med Python og Azure OpenAI

### Spørsmål
- Vi trenger at dte går fortere å kalle på modellen :)
- Hvordan gjør vi det med nøkler? Per nå leser denne fortsatt fra .env-filen da jeg ikke ville pushe de til github.


### Tanker

- Gi mer info i prompt om situasjonen rundt spørreundersøkelsen og hva det har blitt spurt om. Spesielt i situasjonen hvor vi øsnker å oppsummere den generelle viben av feedbacken i hver kategori. Var folk fornøyde? Ønsker de tiltak for forbedring?

# Oppgave
Du jobber i et IT-selskap og har fått i oppgave å analysere svarene fra en intern medarbeiderundersøkelse. Undersøkelsen er anonym, og du har fått tilsendt en CSV-fil med 50 tilbakemeldinger – én per ansatt. Målet er å finne ut hva folk er fornøyde eller misfornøyde med, og særlig se nærmere på temaene Nettverk, Opplæring og IT-support, som ledelsen er ekstra interessert i. Tilbakemeldinger som ikke passer i disse kategoriene skal også få sin plass. Til slutt skal du lage en oppsummering som kan sendes til ledelsen.

For å jobbe effektivt bruker du en språkmodell til å hjelpe deg med både kategorisering og oppsummering. 

**Oppgaven blir dermed å bruke en språkmodell til å kategorisere samt oppsummere tilbakemeldingene fra undersøkelsen.** 


## Datasett

In [ ]:
# Importerer rådataen med tilbakemeldinger, en rad per ansatt. Alle ansatte har svart på undersøkelsen. 

import pandas as pd
enr_path = "../files/forslag_1.csv"
df = pd.read_csv(enr_path)
df.head()

## Using a language model through the API ???

## Using a language model through the API

TODO: Intro til språkmodeller
- Slik kaller man en språkmodell
- Hva er temperaturparameteren

In [ ]:
from langchain_openai import AzureChatOpenAI
from dotenv import find_dotenv, load_dotenv
import os


# Get environment variables
load_dotenv(find_dotenv(), override=True)


llm = AzureChatOpenAI(
    azure_deployment="gpt-4o-mini",
    model=os.environ.get("OPENAI_MODEL_GPT_4O-MINI", default="gpt-4o-mini"),
    temperature=0,
)

reasoning_llm = AzureChatOpenAI(
    azure_deployment="o3-mini",
    model="o3-mini",
    reasoning_effort="medium",
)

In [ ]:
prompt = 'Hei!'
response = llm.invoke(prompt)

In [ ]:
response.content

## Creating a basic chain with Langchain Expression Language (LCEL)

TODO:
- Fordelene med LCEL
- Forklare pipe-operatoren
- Forklare PromptTemplate


Vurdere å droppe?

In [ ]:
from langchain_core.prompts import PromptTemplate
prompt = PromptTemplate.from_template(
    """
    Hi! Please talk like a {role}.
    """
)
prompt

In [ ]:
chain = prompt | llm

chain.invoke({"role": "pirate"})

In [ ]:
from langchain_core.output_parsers import StrOutputParser
chain2 = chain | StrOutputParser()

chain2.invoke({"role": "pirate"})


### Batching and streaming

In [ ]:
chain2.batch(
    [
        {"role": "pirate"},
        {"role": "cowboy"},
        {"role": "ninja"},
    ]
)

In [ ]:
from time import sleep
for chunk in chain2.stream({"role": "pirate"}):
    print(chunk, end="")
    sleep(0.2)

## Simple categorization of each survey reply

In [ ]:
categorize_prompt = PromptTemplate.from_template(
"""
Categorize the following feedback into one of the following categories:
- Network
- Training
- IT-support
- Other

Feedback:
<feedback>
{feedback}
</feedback>
"""
)

categorize_chain = categorize_prompt | llm | StrOutputParser()

categorize_chain.invoke({"feedback": "I am very happy with the IT support I received last week."})

## Structured output

TODO
- Hvorfor trenger man strukturert output
- Forklare hva Pydantic er
- FOrklare hvordan Pydantic brukes for å få strukturert output

### Pydantic 
Pydantic er et Python-bibliotek for datavalidering og datastrukturering. Hovedklassen i Pydantic heter BaseModel og er klassen vi arver fra når vi lager våre egne datamodeller. Når vi arver fra BaseModel får vi automatisk funksjonalitet som kan:
1. Validere innhold du sender inn
    - Eks: Du definerer en liste med godkjente land, "Norge, Sverige, Finland", da vil ikke pydantic godkjenne "Australia".
2. Konvertere data til riktig type
    - Eks: Du definerer at output skal være en int og sender inn '1', pydantic vil da returnere 1 (som int)
3. Påtvinge JSON-formatering
    - Sørger for at responsen LLMen gir matcher JSOM-skjemaet til Pydantic. Dette gjør at vi kan være sikre på strukturen til outputen, som for eksempel er svært nyttig om vi ønsker å bruke outputen fra en modell som input i en annen. 



### Structured output 
I denne konteksten refererer structured output til strategien og verktøyene vi bruker for å forsikre oss om at dataen vår blir organisert på en måte vi definerer på forhånd. 
Funksjonalitet i Pydantic biblioteket lar oss bestemme strukturen på outputen gjennom å spesifisere felter, definere typer (eks. int, str, List[int]) og validere data.
I dette kurset kommer vi til å bruke BaseModel til å lage våre egne klasser for å sikre at det vi mottar fra modellene når vi prompter de komme rpå akkurat den formen vi ønsker. 

In [38]:
from pydantic import BaseModel, Field

class Categorize(BaseModel):
    "Categorization of a single feedback entry from an IT survey."
    category : str = Field(description="The best fitting category. Only one.")


I eksempelet over har vi laget vår egen klasse 'Categorize' som arver av BaseModel.
For variabelen category har vi brukt type hinting ( x: type ) for å definere typen Pydantic skal forvente at category er lik. Dette for ekesempel blir da ulovlig

In [ ]:
Categorize(category=1)

Mens dette er helt ok

In [ ]:
Categorize(category="Network")

Vi bruker også funksjonen Field, denne kan du bruke til å sette standardverdier, valideringsregler og beskrivelser.
I dette kurset bruker vi kun beskrivelse, men for de spesielt interesserte kan dere lese mer om funksjonaliteten [her](https://docs.pydantic.dev/latest/concepts/fields/).

For å få LLMen til å skjønne at den må følge reglene vi har definert i klassen vår bruker vi wrapperen with_structured_output(...). 
Denne wrapper kallet vårt til språkmodellen med logikk som forsikrer at outputen følger strukturen vi har definert i klassen.

In [ ]:
categorize_chain_structured_output = categorize_prompt | llm.with_structured_output(
    Categorize,
    method="json_schema", # Påtvinger JSON-skjema for output
    strict=True           # Modellen må følge skjema etter punkt og prikke, ingen ekstra felter, ingen manglende felter og alle typer må være en eksakt match.
)

categorize_chain_structured_output.invoke(
    {"feedback": "I am very happy with the IT support I received last week."}
)

Vi kan bruke Literal for å definere våre egene typer til å bruke for type hinting. Vi kan type hinte variabler med CATEGORIES under og dette vil da modellen toke på samme måte som at en int bare har lov til å være et heltall har denne "typen" bare lov til å være en av verdiene listet opp i Literal-objektet.


In [ ]:
from typing import Literal

CATEGORIES = Literal[
    "Network",
    "Training",
    "IT-support",
    'Other'
]

La oss sette sammen det vi har lært. Legg merke til at vi ikke lenger lister kategoriene i prompten (som på ingen måte garanterer at vi kun får ut en av kategorierne vi øsnker), men definerer de som et krav til strukturen på outputen fra modellen. 

In [ ]:
from typing import Literal

categorize_prompt2 = PromptTemplate.from_template(
"""
Categorize the following feedback into the provided categories.

Feedback:
<feedback>
{feedback}
</feedback>
"""
)



class CategorizeFromOptions(BaseModel):
    "Categorization of a single feedback entry from an IT survey."
    category: CATEGORIES = Field(
        description="Chosen category for the feedback. Choose 'Other' if the other categories provided are not a good fit."  ## noqa: E501
    )


categorize_chain_structured_output2 = categorize_prompt2 | llm.with_structured_output(
    CategorizeFromOptions,
    method="json_schema",
    strict=True,
)

In [ ]:
result = categorize_chain_structured_output2.invoke(
    {"feedback": "I am very happy with the IT support I received last week."}
)

result


## Evaluate performance across whole dataset

In [ ]:
def categorize_single_feedback(feedback: str) -> str:
    result = categorize_chain_structured_output2.invoke(
        {"feedback": feedback}
    )
    return result.category

df["AI Classification"] = df["Feedback"].apply(
    lambda feedback: categorize_single_feedback(feedback)
)

In [ ]:
df

In [ ]:
(df["AI Classification"] == df["Category"]).value_counts()

#TODO: Replace this be an evaluation function which is imported

In [32]:
others = df[df['AI Classification'] == "Other"].copy()
others

,ID,Category,Feedback,AI Classification
25,26,Security,The new cybersecurity measures give me confide...,Other
27,28,Security,The stringent security protocols inspire trust...,Other
28,29,Security,Enhanced monitoring tools fortify our security...,Other
29,30,Security,"Timely security updates fortify our systems, e...",Other
30,31,Security,I appreciate the proactive approach to securit...,Other
31,32,Security,The security team usually implements robust me...,Other
32,33,Security,Our enhanced firewall settings significantly l...,Other
33,34,Security,Robust security protocols foster a safe system...,Other
34,35,Security,"I value our firm stance on cybersecurity, even...",Other
35,36,Business needs,The IT solutions seem aligned with our evolvin...,Other


In [ ]:
# Denne per nå spytter bare ut "other" igjen, ser på det når jeg kommer tilbake
categorize_prompt_other = PromptTemplate.from_template(
"""
Categorize the following feedback from an IT-survey into the category that best describes the feedback.

Feedback:
<feedback>
{feedback}
</feedback>
"""
)

categorize_chain_structured_output_others = categorize_prompt_other | reasoning_llm.with_structured_output(
    Categorize,
    method="json_schema",
    strict=True,
)

def categorize_single_feedback_other(feedback: str) -> str:
    result = categorize_chain_structured_output_others.invoke(
        {"feedback": feedback}
    )
    return result.category

others["AI Classification"] = df[df['AI Classification'] == "Other"]["Feedback"].apply(
    lambda feedback: categorize_single_feedback(feedback)
)

print("done")

done


In [47]:
df

,ID,Category,Feedback,AI Classification
0,1,Training,The recent training sessions on new software u...,Training
1,2,Training,The interactive training modules are well-desi...,Training
2,3,Training,I appreciate how the sessions cover both the b...,Training
3,4,Training,The hands-on exercises are engaging and boost ...,Training
4,5,Training,The training materials are succinct and creati...,Training
5,6,Training,"Live workshops are full of energy and support,...",Training
6,7,Training,Although the sessions are structured to be int...,IT-support
7,8,Training,"I find the training generally beneficial, yet ...",Training
8,9,Training,The curriculum is robust and adapts to emergin...,Training
9,10,Training,While the training sessions aim to cover a wid...,Training


## Chain-of-thought

Chain of Thought (CoT) er en teknikk innen prompt engineering som hjelper språkmodeller med å løse oppgaver som krever flere tankesteg. I stedet for å hoppe rett til svaret, blir modellen ledet gjennom en logisk og trinnvis prosess, noe som gir mer presise og gjennomtenkte svar – spesielt på komplekse problemer [1]. 

Du kan altså be modellen om å "tenke høyt" under oppgaven og forklare stegene sine før den leverer et endelig svar. Dette ber du om i prompten som sendes inn. 

Eksempel på en prompt **uten** CoT: 

    Prompt: "Hvor mange armer har Eline og Kaspara?"

    Svar: "4"

Eksempel på en prompt **med** CoT:

    Prompt: "Hvor mange armer har ELine og Kaspara? Tenk trinn for trinn."

    Svar: "En person har to armer. To personer betyr 2x2 = 4 armer. Svaret er 4."

Det kan være fordelaktig å bruke CoT når man jobber med komplekse oppgaver, da nøyaktigheten på outputet fra modellen øker når den "får lov" til å jobbe seg gjennom problemet. Dette gir ofte bedre resultater på logiske oppgaver, eller oppgaver med flere steg. 

I tillegg kan du se hvordan modellen tenker, som gjør det lettere for deg å evaluere svaret. Det blir også lettere å se hvor det gikk galt hvis modellen svarer feil.

### *Kilder*
[1] xxxx, Link: https://www.ibm.com/think/topics/chain-of-thoughts 

---
TODO:
- Forklare hva chain-of-thought er og hvorfor det kan være nyttig

In [ ]:
class CategorizeCot(BaseModel):
    "Categorization of a single feedback entry from an IT survey."
    chain_of_thought: str = Field(
        description="Use this space to think through the categorization."
    )
    category: CATEGORIES = Field(
        description="Chosen category for the feedback. Choose 'Other' if the other categories provided are not a good fit."  ## noqa: E501
    )


categorize_chain_cot = categorize_prompt2 | llm.with_structured_output(
    CategorizeCot,
    method="json_schema",
    strict=True,
)

In [ ]:
categorize_chain_cot.invoke(df["Feedback"][0])

In [ ]:
result.category

## Resonneringsmodell
Resonneringsmodeller, som Azure Open AI sin O3-mini-modell, er språkmodeller som er spesielt trent på å tenke før de svarer. Slike modeller vil altså produsere en trinnvis tenkning før det endelige svaret leveres. Resonneringsmodeller er fordelaktige å bruke til oppgaver som krever kompleks problemløsning, logisk tenkning som koding eller matematikk eller til oppgaver med flere steg. De vil også være fordelaktige å bruke i situasjoner der nøyaktighet og forklarbarhet er viktig [2]. 

Dette kan minne om CoT, men det er en viktig forksjell her. CoT er en teknikk du kan bruke med språkmodeller for å gjøre dem bedre til å resonnere. Det er raskt og fleksibelt.
Resonneringsmodeller er som nevnt en egen type modell som passer for oppgaver som krever presis og systematisk tenkning. Du vil få enda mer presise svar med en resonneringsmodell sammenlignet med CoT. 

#### Kilder
[2] xxx, Link: https://platform.openai.com/docs/guides/reasoning?api-mode=chat

In [ ]:
## Kategoriser radene som ligger i *Other*



In [ ]:
# Nå har vi definert passende kategorier for alle tilbakemeldinhgene. 
#Videre vil vi få modellen til å oppsummere per kategori, slik at vi sitter igjen med en overordnet oversikt. 

from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputKeyParser

# Din tur (forslag)
'''Lag en LCEL-kjede som tar resultatet fra forrige oppgave (feedback med kategori) og lager en oppsummering
## per kategori ved hjelp av en LLM. Inkluder structured output.'''

    # Tips: Begynn med en funksjon som lager LCEL-kjeden
def summary_chain(x, y):
    prompt = ___

    # Tips 2: Bruk structured output med method = "function_calling"


# Fasit
## OBS: bytt ut feedback_txt med resultat fra forrige oppgave
def build_summary_chain(struktur, llm_model):
    # 1. Prompt Template
    prompt = PromptTemplate.from_template("""
You are a domain expert in internal IT operations and organizational analysis. You will be provided with a dataset containing qualitative feedback from employees in an IT company. 
Each row in the dataset represents a feedback entry and is associated with a specific category.

For each category, carefully:
1. Read and interpret the feedback entries assigned to that category.
2. Identify core themes, recurring patterns, and contrasting opinions within that category.
3. Evaluate the feedback logically: What are the likely underlying causes of recurring issues or praises? Are there signs of systemic problems, isolated incidents, or misaligned expectations?
4. Summarize each category in 3 to 6 bullet points, highlighting key sentiments (positive and negative), representative concerns or compliments, and any significant outliers

Present your findings in a clean, professional way with one section per category. 

This is the employee feedback data: {feedback_txt}
""")

    # 2. Structured output LLM
    structured_llm = llm_model.with_structured_output(struktur, method="function_calling")

    # 3. LCEL Chain
    chain = prompt | structured_llm

    return chain

chain = build_summary_chain(struktur, llm_model)
response = chain.invoke({"feedback_txt": feedback_txt})


In [ ]:
# Ekstraoppgave
'''Bruk oppsummeringen til å lage en rapport som kan sendes til ledelsen med forslag til endringer for å forbedre
resultatene på neste års undersøkelse'''

Herfra og ned er gammel kode. kan trolig slettes men tørr ikke helt enda. 

In [ ]:
structured_llm = Llm.llm.with_structured_output(   #Gå igjennom denne, hvordan bruker man structured output?
        Categorize, method="function_calling"
    )

response = structured_llm.invoke("Send inn prompt + data")


In [ ]:
# Find potential new categories based on feedback categorized as others

In [ ]:
class CategorizeFromOptions2(BaseModel):
    """Categorization of IT-questionaire feedback."""

    category: Literal[
       # Insert the categories you found while exploring the feedback initially categorized as 'Other'.
    ] = Field(
        description="Categorize the feedback into the most fitting category. If the categories provided are not a perfect fit, default to 'Other'."
    )
    reason : str = Field(description="Why do you believe this category is the most fitting one? Explain.")

### Oppgave??

In [ ]:
from pydantic import BaseModel, Field
from typing import Literal, Optional

# Kan de mekke på disse selv? Hvordan kan vi gjøre dette steget mer interaktivt


class Categorize_search(BaseModel):
    """Categorization of IT-questionaire feedback."""

    categories: Literal[
        "Network",
        "IT Training",
        "IT Support",
        'Other'
    ] = Field(
        description="Categorize the feedback into the most fitting category. If the categories provided are not a perfect fit, default to 'Other'."
    )
    #if_other: str = Field(
        #description="If you chose to categorize the feedback as 'Other', return an explanation as to why and what category you think would be the best fit for the feedback."
    #)


class Categorize_bound(BaseModel):
    # Begrenser kategoriene modellen kan velge mellom. Den får kun lov til å putte feedbacken i en av de forhåndsbestemte kategoriene.
    """Categorization of IT-questionaire feedback."""

    categories: Literal[
        "Cyber security",
        "IT training",
        "IT support",
        "Quality of technology",
        "Data quality",
        "Network",
    ] = Field(description="Categorize the feedback into the most fitting category.")
   # rating: Optional[int] = Field(
       # description="Your certainty of the corectness of the best category on a scale from 1-10"
  #  )
    #reason: str = Field(
    #    description="Give a short scentence as to why you think this category is the best fit."
   # )



class Categorize(BaseModel):
    "Categorization of feedback on IT-services."

    # Thoughts: Forklar tankegangen din.
    cat_1: str = Field(description="The best fitting general category. Only one.")
    
class Summarize(BaseModel):

    # 1. Oppsummer kategori, tar inn alle feedbacks under 1 kategori
    # 2. Likely underlying causes of recurring issues or praises



# Case

# Kategorisering

In [ ]:
# Lag en funksjon som tar inn enten full prompt eller data fra undersøkelsen + pydantic objekt (struct) + modell og returnerer output fra modell

In [ ]:
## KURS
def categorize_feedback(feedback_txt : str, struktur, llm_model) -> dict:
    # Prompt
    task = f"""
    PROMT
    Catgorize the feedback: {feedback_txt}.
    """

    # Defining structured output for the model
    structured_llm = llm_model.with_structured_output(   #Gå igjennom denne, hvordan bruker man structured output?
        struktur, method="function_calling"
    )

    # Prompt model
    response = structured_llm.invoke(task)
    # Return the response
    return response.__dict__

In [ ]:
def categorize_feedback(feedback_txt : str, struktur, llm_model) -> dict:
    # Prompt
    task = f"""
    The following feedback is from an internal survey at 'IT and Things Company' where they asked their employees for feedback on their IT-services in general.
    Catgorize the feedback: {feedback_txt}.
    """

    # Defining structured output for the model
    structured_llm = llm_model.with_structured_output(
        struktur, method="function_calling"
    )

    # Prompt model
    response = structured_llm.invoke(task)
    # Return the response
    return response.__dict__

# Din tur

In [ ]:
# Returner en dataframe med kategoriserte feedback 
# 1. Be den kategorisere uten begrensninger
# 2. Gi den et sett med forhåndsdefinerte kategorier
# 3. La 'other' være et alternativ. 

In [ ]:
## returnerer kategorisert feedback

responses = []

for f in FeedbackData.df["Feedback"]:
    feedback_4O = categorize_feedback(feedback_txt = f, struktur = Categorize_search, llm_model = Llm.llm)
    feedback_3O = categorize_feedback(feedback_txt = f, struktur = Categorize_search, llm_model = LlmRes.llm)
    responses.append({'Feedback': f, 'Category_4O': feedback_4O['categories'], 'Category_3O': feedback_3O['categories']})
    #print(f)

    
cat_df = pd.DataFrame(responses)
print(cat_df)

In [ ]:
# Identifiser de som ble identifisert som 'other' og finn kategori for de. 

In [ ]:
others_df = cat_df[(cat_df['Category'] == 'Other') | (cat_df['Category2'] == 'Other')]
print(others_df)

In [ ]:
## returnerer kategorisert feedback

# Heller send inn alle others-feedback samtidig og be den finne X nye kategorier. 
responses_new_cat = []

for f in others_df["Feedback"]:
    output_i = categorize_feedback(feedback_txt = f, struktur = Categorize, llm_model = Llm.llm)
    responses_new_cat.append({'Feedback': f,'Category': output_i["cat_1"]})

new_cat_df = pd.DataFrame(responses_new_cat)
print(new_cat_df)

In [ ]:
# Ny kategorisering hvor vi gir den grunnkategorier + de modellen har gjenkjent under other 

# Reasoning models - when we want the LLM to return logic analyzation across rows

Nå vil vi teste resoneringsmodellen, sammenlikne?

## Summarization per category

- Få modellen til å oppsummere per kategori.

In [ ]:
# Now that we have defined fitting categories for the feedback, we can use a reasoning model to create a summary of the rows per category to provide an overall overview
def summarize(feedback_txt : str, struktur, llm_model) -> dict:

    # Prompt
    task = f"""
You are a domain expert in internal IT operations and organizational analysis. You will be provided with a dataset containing qualitative feedback from employees in an IT company. 
Each row in the dataset represents a feedback entry and is associated with a specific category.

For each category, carefully:
1. Read and interpret the feedback entries assigned to that category.
2. Identify core themes, recurring patterns, and contrasting opinions within that category.
3. Evaluate the feedback logically: What are the likely underlying causes of recurring issues or praises? Are there signs of systemic problems, isolated incidents, or misaligned expectations?
4. Summarize each category in 3 to 6 bullet points, highlighting key sentiments (positive and negative), representative concerns or compliments, and any significant outliers

Present your findings in a clean, professional way with one section per category. 

This is the employee feedback data: {feedback_txt}
    """

    # Defining structured output for the model
    structured_llm = llm_model.with_structured_output(
        struktur, method="function_calling"
    )

    # Prompt model
    response_sum = structured_llm.invoke(task)
    # Return the response
    return response_sum.__dict__

# Print head of response


## Generating a report for the leadership

- Bruk oppsummeringen til å generere en en rapport til ledelsen med forlag til forbedringspotensiale i IT. Hvilke tiltak bør bedriften gjøre?

In [ ]:
def report(feedback_txt : str, struktur, llm_model) -> dict:

    # Prompt
    task = f"""
You are an expert HR and technical operations analyst. I will provide you with a dataset of employee feedback collected from an IT company.

Your task is to deeply analyze this feedback and generate a concise executive-level summary report in markdown format that includes:

1. Key Takeaways
Provide a short summary of the overall feedback in 3-5 bullet points. Focus only on the main issues or areas of satisfaction.
Include both positive and negative themes, but prioritize the most important and impactful points.
Limit each point to 1-2 sentences.
Before finalizing each point, take a moment to reflect on why each issue might be present (e.g., systemic problems, temporary issues, resource constraints, etc.)

2. Suggested Improvements
Based on the overall feedback, propose 2-3 high-level, actionable measures that the company could take to address the most pressing issues and enhance overall performance or satisfaction.
Each suggestion should be brief, directly tied to the feedback, and strategic in nature.
Think about short-term vs long-term solutions and consider the feasibility of each suggestion.

3. Output
Present your findings in a structured way with clear section headings, bullet points for easy scanning, and a consise, direct and professional tone suitable for leadership review.

This is the employee feedback data: {feedback_txt}
    """

    # Defining structured output for the model
    structured_llm = llm_model.with_structured_output(
        struktur, method="function_calling"
    )

    # Prompt model
    response = structured_llm.invoke(task)
    # Return the response
    return response.__dict__

1. Skulle man hatt en oppgave til slutt for de som er fort ferdig hvor de sender inn hele undersøkelsen og ber modellen velge beste kategorier før vi igjen bruker de kategoriene i structured output for å kategorisere. kjør så hele på nytt og se om du synes resultatene ble bedre. 

2. Hvordan kunne dette vært gjort bedre? Be LLMen si hva som kunne vært gjort annerledes for å skape mer verdi. 
